In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
from decompose_essential_matrix import *

In [2]:
def correspondingFeatureDetection(img1, img2):

    orb = cv2.ORB_create()
    Keypoints1, Descriptors1 = orb.detectAndCompute(img1, None)
    Keypoints2, Descriptors2 = orb.detectAndCompute(img2, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors.
    matches = bf.match(Descriptors1,Descriptors2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    kp1_list = np.mat([])
    kp2_list = np.mat([])
    k = 0

    number_of_matches = 150

    for m in matches:
        img1Idx = m.queryIdx
        img2Idx = m.trainIdx

        (img1x, img1y) = Keypoints1[img1Idx].pt
        (img2x, img2y) = Keypoints2[img2Idx].pt

        if k == 0:
            kp1_list = [[img1x,img1y,1]]
            kp2_list = [[img2x,img2y,1]]
            k = 1
        else:
            kp1_list = np.append(kp1_list,[[img1x,img1y,1]],axis = 0)
            kp2_list = np.append(kp2_list,[[img2x,img2y,1]],axis = 0)
            k+=1
        if k == number_of_matches:
            break
    return kp1_list,kp2_list

In [3]:
def F_matrix(image_coords_1, image_coords_2):
    '''
    image_coords_1  : N*3 homogeneous coordinates of image pixels    
    image_coords_2  : N*3 homogeneous coordinates of image pixels
    return          : Fundamental Matrix of dimension 3*3
    '''
    
    A = np.zeros((len(image_coords_1),9))

    # Constructing A matrix of size N*9
    for i in range(len(image_coords_1)):
        A[i,:] = np.kron(image_coords_1[i,:], image_coords_2[i,:])
        
    u, s, vh = np.linalg.svd(A, full_matrices=True)
    F = np.reshape(vh[8,:], (3,3))
    uf, sf, vhf = np.linalg.svd(F, full_matrices=True)
    
    F = uf @ np.diag(np.array([sf[0], sf[1], 0])) @ vhf

    return F

In [4]:
def F_RANSAC(image_points_1, image_points_2, threshold, n_iters):
    '''
        image_points_1  : N*3 matrix of a normalized 2D homogeneous of image 1 
        image_points_2  : N*3 matrix of a normalized 2D homogeneous of image 2
        threshold       : Inlier threshold
        n_iters         : Number of Iterations 
        return          : Fundamental Matrix of dimension 3*3
    '''

    n = 8 
    F = np.zeros((3,3))
    max_inliers = -9999999

    for i in range(n_iters):

        # Randomly sample 8 matching points from image 1 and image 2
        indices = np.random.choice(image_points_1.shape[0], n, replace=False)  
        matched_points_1 = image_points_1[indices] 
        matched_points_2 = image_points_2[indices]

        # Get the F matrix for the current iteration
        F_current = F_matrix(matched_points_1, matched_points_2)

        # Counting the number of inliers 
        number_current_inliers = 0
        for i in range(len(image_points_1)):
            error = np.abs(image_points_2[i,:] @ F_current @ image_points_1[i,:].T)
            if error < threshold:
                number_current_inliers += 1

        # Updating F matrix
        if number_current_inliers > max_inliers:
            max_inliers = number_current_inliers
            F = F_current
    return F

In [5]:
def NormalizationMat(image_coords):
    '''
        image_coords : N*3 matrix of image coordinates
        return       : 3*3 Scale Transformation matrix 
    '''

    mu = np.mean(image_coords,axis = 0)
    d = 0
    for i in range(len(image_coords)):
        d = d + np.sqrt((image_coords[i,0] - mu[0])**2 + (image_coords[i,1] - mu[1])**2)

    d = d/i
    T = np.mat([
        [1.44/d, 0, -1.44 * mu[0]/d], 
        [0, 1.44/d, -1.44 * mu[1]/d],
        [0,      0,               1]
        ])
    
    return T

In [7]:
if __name__ == "__main__":
    ground_truth = np.loadtxt('../mr19-assignment2-data/ground-truth.txt')
    ground_truth_translation = np.concatenate((np.concatenate((np.array([ground_truth[:,3]]).T, np.array([ground_truth[:,7]]).T), axis=1), np.array([ground_truth[:,11]]).T ), axis=1)
    for i in range(800, 0, -1):
        ground_truth_translation[i] = ground_truth_translation[i] - ground_truth_translation[i-1]
    ground_truth_norm = np.linalg.norm(ground_truth_translation, axis=1)

    C = np.concatenate((np.eye(3), np.zeros((3,1))), axis = 1) 
    C = np.concatenate((C, np.array([[0, 0, 0, 1]])), axis = 0)
    
    K  = np.array([[7.215377e+02, 0.000000e+00, 6.095593e+02],[0.000000e+00, 7.215377e+02, 1.728540e+02],[0.000000e+00, 0.000000e+00, 1.000000e+00]])
    
    f = open('results.txt','wb')
    K_inverse = np.linalg.inv(K)

    cumulative_translation = np.zeros((3,1))
    cumulative_orientation = np.eye(3)
    initial_matrix = np.concatenate((cumulative_translation, cumulative_orientation), axis=1)
    np.savetxt(f,np.reshape(initial_matrix, (1,12)))

    dirFiles = os.listdir('../mr19-assignment2-data/images/')
    for i in range(len(dirFiles)):
        dirFiles[i] = dirFiles[i].split(".")[0]

    dirFiles.sort(key=float)
    for i in range(len(dirFiles)):
        dirFiles[i] = '../mr19-assignment2-data/images/' + dirFiles[i] + '.png'

    key_point_1 = np.zeros((800,150,3))
    key_point_2 = np.zeros((800,150,3))

    for i in range(1,len(dirFiles)):
        print("Iteration {}".format(i))
        img1 = cv2.imread(dirFiles[i-1])
        img2 = cv2.imread(dirFiles[i])

        kp1, kp2 = correspondingFeatureDetection(img1, img2)
        key_point_1[i-1] = kp1
        key_point_2[i-1] = kp2



    for i in range(1, len(dirFiles)):
        print("Iteration {}".format(i))


        kp1 = key_point_1[i-1]
        kp2 = key_point_2[i-1]
        if i > 1:
            kpPrev = key_point_2[i - 2]

        T1 = NormalizationMat(kp1)
        T2 = NormalizationMat(kp2)

        points1 = T1 @ kp1.T
        points2 = T2 @ kp2.T

        F = F_RANSAC(points1.T, points2.T, 0.005, 300)
        FundamentalMatrix = T2.T @ F @ T1
        E = compute_essential_matrix(FundamentalMatrix, K)
        
        
        Transformation_info = cv2.recoverPose(E, (kp1[:,0:2]), (kp2[:,0:2]), K)
        Rotation = Transformation_info[1].T
        Translation = -Transformation_info[2]
        Translation = Translation / np.linalg.norm(Translation) * ground_truth_norm[i]
        print("Translation: ",Translation)

        Transformation = np.concatenate((Rotation, Translation), axis = 1)
        Transformation = np.concatenate((Transformation, np.array([[0, 0, 0, 1]])), axis = 0)
        C = C @ Transformation

        print("C : ", C)
        Reshaped = np.reshape(C[0:3,:],(1,12))
        np.savetxt(f, Reshaped)

800 - 799
799 - 798
798 - 797
797 - 796
796 - 795
795 - 794
794 - 793
793 - 792
792 - 791
791 - 790
790 - 789
789 - 788
788 - 787
787 - 786
786 - 785
785 - 784
784 - 783
783 - 782
782 - 781
781 - 780
780 - 779
779 - 778
778 - 777
777 - 776
776 - 775
775 - 774
774 - 773
773 - 772
772 - 771
771 - 770
770 - 769
769 - 768
768 - 767
767 - 766
766 - 765
765 - 764
764 - 763
763 - 762
762 - 761
761 - 760
760 - 759
759 - 758
758 - 757
757 - 756
756 - 755
755 - 754
754 - 753
753 - 752
752 - 751
751 - 750
750 - 749
749 - 748
748 - 747
747 - 746
746 - 745
745 - 744
744 - 743
743 - 742
742 - 741
741 - 740
740 - 739
739 - 738
738 - 737
737 - 736
736 - 735
735 - 734
734 - 733
733 - 732
732 - 731
731 - 730
730 - 729
729 - 728
728 - 727
727 - 726
726 - 725
725 - 724
724 - 723
723 - 722
722 - 721
721 - 720
720 - 719
719 - 718
718 - 717
717 - 716
716 - 715
715 - 714
714 - 713
713 - 712
712 - 711
711 - 710
710 - 709
709 - 708
708 - 707
707 - 706
706 - 705
705 - 704
704 - 703
703 - 702
702 - 701
701 - 700


Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration 77
Iteration 78
Iteration 79
Iteration 80
Iteration 81
Iteration 82
Iteration 83
Iteration 84
Iteration 85
Iteration 86
Iteration 87
Iteration 88
Iteration 89
Iteration 90
Iteration 91
Iteration 92
Iteration 93
Iteration 94
Iteration 95
Iteration 96
Iteration 97
Iteration 98
Iteration 99
Iteration 100
Iteration 101
Iteration 102
Iteration 103
Iteration 104
Iteration 105
Iteration 106
Iteration 107
Iteration 108
Iteration 109
Iteration 110
Iteration 111


Iteration 631
Iteration 632
Iteration 633
Iteration 634
Iteration 635
Iteration 636
Iteration 637
Iteration 638
Iteration 639
Iteration 640
Iteration 641
Iteration 642
Iteration 643
Iteration 644
Iteration 645
Iteration 646
Iteration 647
Iteration 648
Iteration 649
Iteration 650
Iteration 651
Iteration 652
Iteration 653
Iteration 654
Iteration 655
Iteration 656
Iteration 657
Iteration 658
Iteration 659
Iteration 660
Iteration 661
Iteration 662
Iteration 663
Iteration 664
Iteration 665
Iteration 666
Iteration 667
Iteration 668
Iteration 669
Iteration 670
Iteration 671
Iteration 672
Iteration 673
Iteration 674
Iteration 675
Iteration 676
Iteration 677
Iteration 678
Iteration 679
Iteration 680
Iteration 681
Iteration 682
Iteration 683
Iteration 684
Iteration 685
Iteration 686
Iteration 687
Iteration 688
Iteration 689
Iteration 690
Iteration 691
Iteration 692
Iteration 693
Iteration 694
Iteration 695
Iteration 696
Iteration 697
Iteration 698
Iteration 699
Iteration 700
Iteration 701
Iterat

Translation:  [[0.27726277]
 [0.05130993]
 [0.8462054 ]]
C :  [[ 9.91272945e-01  1.15545504e-01 -6.34601037e-02  1.19338369e-01]
 [-1.15177288e-01  9.93300070e-01  9.44259939e-03  1.26562663e-02]
 [ 6.41259753e-02 -2.05103069e-03  9.97939704e-01  1.74630229e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 20
Translation:  [[-0.11251689]
 [-0.01886941]
 [ 0.88488307]]
C :  [[ 9.91549338e-01  1.15712195e-01 -5.86566106e-02 -5.05316220e-02]
 [-1.15284294e-01  9.93275566e-01  1.06387046e-02  1.52282672e-02]
 [ 5.94932059e-02 -3.78661458e-03  9.98221529e-01  1.83389063e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 21
Translation:  [[-0.08932962]
 [-0.01279124]
 [ 0.88304173]]
C :  [[ 9.91905322e-01  1.13920509e-01 -5.60887638e-02 -1.92382688e-01]
 [-1.13504437e-01  9.93481378e-01  1.05591253e-02  2.22157686e-02]
 [ 5.69260432e-02 -4.10732898e-03  9.98369949e-01  1.92151115e+01]
 [ 0.00000000e+00  0.00000000e+00  0.000000

Translation:  [[-0.17910139]
 [ 0.011337  ]
 [ 0.65279691]]
C :  [[ 9.90624165e-01 -2.58201965e-02 -1.34153202e-01 -1.87050692e+00]
 [ 3.20511864e-02  9.98495263e-01  4.44964099e-02  3.12077390e-01]
 [ 1.32802431e-01 -4.83789881e-02  9.89961104e-01  3.51867943e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 45
Translation:  [[-0.17630057]
 [ 0.01250437]
 [ 0.63915948]]
C :  [[ 9.92502586e-01  6.58666989e-03 -1.22046026e-01 -2.13122268e+00]
 [-2.74889505e-03  9.99497225e-01  3.15870410e-02  3.47352601e-01]
 [ 1.22192718e-01 -3.10147282e-02  9.92021686e-01  3.57955192e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 46
Translation:  [[-7.29405925e-02]
 [-7.28983439e-05]
 [ 6.49031065e-01]]
C :  [[ 9.92954742e-01  2.16451087e-03 -1.18474450e-01 -2.28282855e+00]
 [ 1.09288820e-03  9.99623331e-01  2.74226558e-02  3.67981216e-01]
 [ 1.18489181e-01 -2.73589355e-02  9.92578361e-01  3.64304615e+01]
 [ 0.00000000e+00  0.0000000

...

Translation:  [[ 0.05298212]
 [-0.03617256]
 [ 0.7093116 ]]
C :  [[ 6.20811291e-01 -2.85811203e-02  7.83438868e-01  2.37186422e+02]
 [-2.86780801e-01  9.21791207e-01  2.60878791e-01  1.25402651e+02]
 [-7.29623268e-01 -3.86631725e-01  5.64061872e-01  2.94642233e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 692
Translation:  [[ 0.07520197]
 [-0.0407054 ]
 [ 0.69782574]]
C :  [[ 6.24086138e-01 -3.35494036e-02  7.80634953e-01  2.37780976e+02]
 [-2.80402612e-01  9.22911522e-01  2.63834605e-01  1.25525611e+02]
 [-7.29308486e-01 -3.83547599e-01  5.66568947e-01  2.94996718e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 693
Translation:  [[ 0.0482327 ]
 [-0.06260548]
 [ 0.69568859]]
C :  [[ 6.27678387e-01 -3.69858114e-02  7.77593655e-01  2.38356256e+02]
 [-2.78999729e-01  9.21828465e-01  2.69056560e-01  1.25637854e+02]
 [-7.26759241e-01 -3.85829407e-01  5.68292772e-01  2.95379710e+02]
 [ 0.00000000e+00  0.00000000e+00  0.0

Translation:  [[-0.19565898]
 [-0.18535346]
 [ 0.36254938]]
C :  [[-7.36681320e-01 -8.86256670e-02 -6.70407431e-01  2.35742696e+02]
 [ 2.40279584e-01 -9.60988340e-01 -1.36993184e-01  1.24854839e+02]
 [-6.32112612e-01 -2.62005538e-01  7.29237097e-01  3.00788977e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 716
Translation:  [[ 0.12878448]
 [-0.012594  ]
 [ 0.42148568]]
C :  [[-7.38380630e-01 -9.03192203e-02 -6.68308673e-01  2.35366372e+02]
 [ 2.45441159e-01 -9.59018832e-01 -1.41568063e-01  1.24840145e+02]
 [-6.28134287e-01 -2.68561571e-01  7.30288984e-01  3.01018233e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 717
Translation:  [[ 0.29637792]
 [ 0.02705018]
 [-0.31523898]]
C :  [[-7.34982667e-01 -1.04410508e-01 -6.69999198e-01  2.35355766e+02]
 [ 2.57062591e-01 -9.57223301e-01 -1.32824603e-01  1.24931575e+02]
 [-6.27470560e-01 -2.69855511e-01  7.30382571e-01  3.00594588e+02]
 [ 0.00000000e+00  0.00000000e+00  0.0

Translation:  [[ 0.17478011]
 [-0.02368563]
 [ 0.22298541]]
C :  [[-1.30443042e-01  3.34541188e-01  9.33309598e-01  2.34233408e+02]
 [ 2.16033266e-01  9.28324651e-01 -3.02560689e-01  1.24558415e+02]
 [-9.67633319e-01  1.62158984e-01 -1.93365517e-01  3.02758841e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 740
Translation:  [[0.08516627]
 [0.02268393]
 [0.28432668]]
C :  [[-1.09517111e-01  3.38660372e-01  9.34513325e-01  2.34495252e+02]
 [ 2.21220633e-01  9.24892771e-01 -3.09248758e-01  1.24511846e+02]
 [-9.69054918e-01  1.72865599e-01 -1.76210246e-01  3.02625131e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 741
Translation:  [[ 0.22847098]
 [-0.03507095]
 [ 0.18496886]]
C :  [[-1.02256052e-01  3.23752382e-01  9.40599859e-01  2.34631209e+02]
 [ 2.04878151e-01  9.32138760e-01 -2.98567037e-01  1.24472750e+02]
 [-9.73431376e-01  1.62178074e-01 -1.61646619e-01  3.02365074e+02]
 [ 0.00000000e+00  0.00000000e+00  0.0000

Translation:  [[0.15503011]
 [0.00309155]
 [0.23003068]]
C :  [[ 1.69038468e-01 -3.22044412e-01 -9.31511349e-01  2.33720914e+02]
 [ 2.78700583e-01 -8.90921784e-01  3.58586615e-01  1.24954311e+02]
 [-9.45384568e-01 -3.20227688e-01 -6.08460878e-02  3.01478826e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 764
Translation:  [[ 0.08606616]
 [-0.00290677]
 [ 0.27375988]]
C :  [[ 1.79105743e-01 -3.20420086e-01 -9.30189282e-01  2.33481389e+02]
 [ 2.74036899e-01 -8.91811983e-01  3.59965505e-01  1.25079054e+02]
 [-9.44894127e-01 -3.19378076e-01 -7.19217215e-02  3.01381734e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 765
Translation:  [[0.10341698]
 [0.00147951]
 [0.26802928]]
C :  [[ 1.88930626e-01 -3.20555882e-01 -9.28196717e-01  2.33250119e+02]
 [ 2.69556671e-01 -8.91981594e-01  3.62916019e-01  1.25202556e+02]
 [-9.44269252e-01 -3.18767568e-01 -8.21146645e-02  3.01264267e+02]
 [ 0.00000000e+00  0.00000000e+00  0.0000000

Translation:  [[-0.06212473]
 [-0.00232918]
 [-0.07177738]]
C :  [[-2.32969203e-02 -3.82474862e-01 -9.23672146e-01  2.31419008e+02]
 [ 3.00967784e-01 -8.83743980e-01  3.58350347e-01  1.25836774e+02]
 [-9.53349698e-01 -2.69647100e-01  1.35701125e-01  3.01151846e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 788
Translation:  [[ 0.08913463]
 [-0.00114448]
 [-0.00719264]]
C :  [[-2.55365219e-02 -4.01576011e-01 -9.15469603e-01  2.31424012e+02]
 [ 3.03611531e-01 -8.75626850e-01  3.75629681e-01  1.25862035e+02]
 [-9.52453634e-01 -2.68354852e-01  1.44283571e-01  3.01066202e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Iteration 789
Translation:  [[0.03202941]
 [0.02790027]
 [0.07262472]]
C :  [[-6.75768786e-01 -2.20868975e-01 -7.03244939e-01  2.31345505e+02]
 [-1.96387467e-01  9.73516235e-01 -1.17038899e-01  1.25874609e+02]
 [ 7.10470626e-01  5.90172576e-02 -7.01247782e-01  3.01038687e+02]
 [ 0.00000000e+00  0.00000000e+00  0.0000